In [1]:
import numpy as np
import sympy as sp
from sympy.abc import x, y
import matplotlib.pyplot as plt
import pickle
from datetime import datetime
from functools import reduce
from scipy.optimize import fsolve, minimize
from IPython.display import clear_output

In [2]:
#Some code that implements progress bars for long computations
#https://www.mikulskibartosz.name/how-to-display-a-progress-bar-in-jupyter-notebook/
bar_length = 20
def update_progress(progress):
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [3]:
with open("data/NE_3-14.pk", 'rb') as file:
    NE_data = pickle.load(file)

def f1(n):
    a, b, c = (0.06672678, 0.29865376, -0.79503788)
    return a + np.exp(-b*n + c)

# for n in range(N, N+1):
#     print("N = " + str(n))
#     print("\tc0 = " + str(NE_data[n][0]))
#     print("\t   ~ {}".format(f1(n)))
#     for m in range(1, n+1):
#         print("\tp{} = {}".format(m, NE_data[n][m]))

In [4]:
N_min = 3
for N in range(N_min, N_min):
    p = sp.IndexedBase('p')
    i,j = sp.symbols('i j', cls=sp.Idx, range=(1,N))
    Z0, c0 = sp.symbols('Z0, c0')
    Z0 = sp.summation(p[i], (i, 1, N))**(N-1)
    L = lambda i: lambda Q: p[i]*sp.diff(Q, p[i]).subs(p[i], 0)
    OneMinusL = lambda i: lambda Q: Q - L(i)(Q)

    Z_memo = dict()
    def Z(i):
        if i == 0:
            return Z0
        if i in Z_memo.keys():
            return Z_memo[i]
        Z_memo[i] = OneMinusL(i)(Z(i-1))
        return Z_memo[i]

    c_memo = dict()
    def c(i):
        if i in c_memo.keys():
            return c_memo[i]
        c_memo[i] = Z(i-1).subs(p[i], 0)
        return c_memo[i]

    c0s = np.linspace(0, NE_data[N-1][0] if N > N_min and N !=5 else .3, 100)
    errors = []
    for c0_init in c0s:
        ps = {c0: c0_init}
        for i in range(1, N):
            ci = c(i).subs(p[N], 1 - sp.summation(p[j], (j, 1, N-1)))
            eq = ci - c0
            try:
                #soln = sp.nsolve(eq.subs(ps).subs(p[i], x), 0)
                soln = fsolve(lambda x: [sp.lambdify([p[i]], eq.subs(ps))(*x)], 0)[0]
                ps[p[i]] = soln
            except:
                ps[p[i]] = sp.nan
        error = (c(N) - c0).subs(ps)**2
        errors.append(error)
        update_progress((list(c0s).index(c0_init)+1) / len(c0s))

    errors = np.array(errors)
    nan_mask = [x != sp.nan for x in errors]
    plt.clf()
    plt.plot(c0s[nan_mask], errors[nan_mask], '.')
    plt.plot(c0s[np.logical_not(nan_mask)], np.zeros(len(c0s[np.logical_not(nan_mask)])), 'r.')
    plt.plot([NE_data[N][0]], [0], 'b.')
    plt.xlim(0, c0s[-1])
    #plt.savefig("error{}.png".format(N))
    print(len(c0s[np.logical_not(nan_mask)]))

In [5]:
errors = np.array(errors)
nan_mask = [x != sp.nan for x in errors]
plt.plot(c0s[nan_mask], errors[nan_mask], '.')
plt.plot(c0s[np.logical_not(nan_mask)], np.zeros(len(c0s[np.logical_not(nan_mask)])), 'r.')
plt.plot([NE_data[N][0]], [0], 'b.')
plt.xlim(c0_min, c0_max)
plt.show()
print(len(c0s[np.logical_not(nan_mask)]))

NameError: name 'errors' is not defined

In [6]:
c0_data = dict()
test_ps = []
for N in range(3, 6):
    start_time = datetime.now()
    print("N = ", N)
    print("Start Time: ", start_time)

    p = sp.IndexedBase('p')
    i,j = sp.symbols('i j', cls=sp.Idx, range=(1,N))
    Z0, c0 = sp.symbols('Z0, c0')
    Z0 = sp.summation(p[i], (i, 1, N))**(N-1)
    L = lambda i: lambda Q: p[i]*sp.diff(Q, p[i]).subs(p[i], 0)
    OneMinusL = lambda i: lambda Q: Q - L(i)(Q)

    Z_memo = dict()
    def Z(i):
        if i == 0:
            return Z0
        if i in Z_memo.keys():
            return Z_memo[i]
        Z_memo[i] = OneMinusL(i)(Z(i-1))
        return Z_memo[i]

    c_memo = dict()
    def c(i):
        if i in c_memo.keys():
            return c_memo[i]
        c_memo[i] = Z(i-1).subs(p[i], 0)
        return c_memo[i]

    def error_func(initial_c0):
        ps = {c0: float(initial_c0)}
        for i in range(1, N):
            ci = c(i).subs(p[N], 1 - sp.summation(p[j], (j, 1, N-1)))
            eq = ci - c0
            soln = fsolve(lambda x: np.array([float(sp.lambdify([p[i]], eq.subs(ps))(*x))], dtype=np.float64), 0)[0]
            ps[p[i]] = float(soln)
        delta = (c(N) - c0).subs(ps).simplify()
        error = np.real(np.abs(delta**2))
        return float(error)

    res = minimize(error_func, 1 if N == 3 else c0_data[N-1], bounds=[(0, 1 if N == 3 or N == 5 else c0_data[N-1])])
    c0_data[N] = res.x[0]
    print(res)
    if N in NE_data.keys():
        foo = res.x[0]
        bar = NE_data[N][0]
        print(foo, bar, (bar-foo)/bar)
    
    end_time = datetime.now()
    print("End Time: ", end_time)
    diff = end_time - start_time
    print("Elapsed Time: {} days {} hours {} minutes {} seconds {} milliseconds {} microseconds".format(diff.days, diff.seconds // 3600, (diff.seconds // 60) % 60, diff.seconds % 60, diff.microseconds // 1000, diff.microseconds % 1000))
    print()

N =  3
Start Time:  2024-05-02 19:43:43.977272


C:\Users\Sean\AppData\Local\Temp\ipykernel_2800\1554770450.py:36: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  soln = fsolve(lambda x: np.array([float(sp.lambdify([p[i]], eq.subs(ps))(*x))], dtype=np.float64), 0)[0]


NameError: name 'ps' is not defined

In [ ]:
# Gradient decent

initial_c0 = NE_data[N-1][0]
num_iter = 10
c0_results = []
error_results = []
for iter in range(num_iter):
    ps = {c0: initial_c0}
    derivs = dict()
    for i in range(1, N):
        ci = c(i).subs(p[N], 1 - sp.summation(p[j], (j, 1, N-1)))
        eq = ci - c0
        #soln = sp.nsolve(eq.subs(ps).subs(p[i], x), 0)
        soln = fsolve(lambda x: [sp.lambdify([p[i]], eq.subs(ps))(*x)], 0)[0]
        ps[p[i]] = soln
        pdvs = dict()
        for l in range(i):
            tmp_ps = dict(ps)
            del tmp_ps[p[i]]
            del tmp_ps[(c0 if l==0 else p[l])]
            tmp_eq = eq.subs(tmp_ps)
            sub = {p[i]: y, (c0 if l==0 else p[l]): x}
            inv_sub = {v:ps[k] for k,v in sub.items()}
            pdvs[l] = sp.idiff(tmp_eq.subs(sub), y, x).subs(inv_sub)
        derivs[i] = pdvs[0] + sum([derivs[k]*pdvs[k] for k in range(1, i)])
    delta = (c(N) - c0).subs(ps).simplify()
    cN_prime = sum([sp.diff(c(N), p[i]).subs(ps)*derivs[i] for i in range(1, N)])
    error = np.real(np.abs(delta**2))
    dE = (2*(cN_prime - 1)*delta)
    y_int = error - dE*initial_c0
    c0_results.append(initial_c0)
    error_results.append(error)
    initial_c0 = -y_int / dE
    update_progress((iter+1)/num_iter)
    

In [ ]:
plt.plot(c0s, errors)
plt.plot([NE_data[N][0]], [0], 'r.')
for c,e in zip(c0_results, error_results):
    plt.plot([c], [e], 'g.')
plt.show()

In [ ]:
foo = NE_data[N][0]
bar = c0_results[-1]
print(foo, bar, (foo-bar)/foo)